HW 3 of Computational methods 540

Review of basic statistics:MVN

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib as mpl
import scipy.stats as st
from scipy.stats import *

In [14]:
N = 2000
d = 3
x = np.random.normal(size=(d, N))
mu3 = np.random.random(3)
C = np.random.random((3, 3))
cov3 = C @ C.T # positive semi-definite

In [15]:
mu3 , cov3

(array([0.07016291, 0.57533288, 0.39547161]),
 array([[0.53864383, 0.2472876 , 0.27038971],
        [0.2472876 , 0.20232173, 0.13264607],
        [0.27038971, 0.13264607, 0.13699239]]))

In [16]:
eigv3 = np.linalg.eigvals(cov3)
L = np.linalg.cholesky(cov3)

In [17]:
eigv3, L

(array([8.05175293e-01, 7.24172104e-02, 3.65451169e-04]),
 array([[0.73392359, 0.        , 0.        ],
        [0.33693916, 0.29798277, 0.        ],
        [0.3684167 , 0.0285656 , 0.0211076 ]]))

array([[ 1.        , -0.90866959, -0.93974699],
       [-0.90866959,  1.        ,  0.996656  ],
       [-0.93974699,  0.996656  ,  1.        ]])

In [2]:
# x = np.linspace(0, 5, 10)
# mean = np.array([5,1,2])
# cov = np.array([[20,11,-2], [11,2,8],[-2,8,7]])
# mvn3 = multivariate_normal(x,mean,cov)

In [1]:
# d = 3
# mean = np.matrix([5,1,2])
# cov = np.matrix([[20,11,-2], [11,2,8],[-2,8,7]])
# # eval = np.linalg.eigvals(cov)
# L = linalg.cholesky(cov)
# n = 50  # Samples to draw
# X = np.random.normal(size=(d, n))
# # Apply the transformation
# Y = L.dot(X) + mean
# eval = np.linalg.eigvals(cov)
# eval

array([-7.68816417, 25.37549149, 11.31267269])

(array([0.64103298, 0.59192331, 0.41129166]),
 array([[1.19540972, 1.30570592, 1.29346098],
        [1.30570592, 1.50171879, 1.39564807],
        [1.29346098, 1.39564807, 2.09241537]]))